<a href="https://colab.research.google.com/github/kiransoorya/Prodigy_Infotech/blob/main/copy_of_stable_diffusion_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **SETUP**

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate invisible_watermark mediapy omegaconf gradio

In [ ]:
!pip install omegaconf

#AKAN MERESTART RUNTIME, KLIK RESTART
#WILL RESTART RUNTIME, JUST CLICK RESTART

In [ ]:
use_refiner = False

## **MODELS**

**EN : JUST CLICK RUN CELL AND ALL WILL BE AUTOMATED**

**ID :KLIK RUN PADA CELL, MAKA SEMUA AKAN BERJALAN OTOMATIS**

In [ ]:
#@title Setup Model
# @markdown IF U WANT TO USE ANOTHER MODEL, GO TO [HERE](https://huggingface.co/models?pipeline_tag=text-to-image&sort=trending), MAKE SURE THE MODEL HAVE A SAFETENSORS FILE

import mediapy as media
import random
import sys
import torch

from diffusers import DiffusionPipeline

model = "digiplay/Realisian_v5" # @param ["digiplay/Realisian_v5", "stabilityai/stable-diffusion-xl-base-1.0", "hakurei/waifu-diffusion", "Linaqruf/anything-v3.0"]

pipe = DiffusionPipeline.from_pretrained(
    model,
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
)

pipe = pipe.to("cuda")  # If you have a GPU for acceleration


##**PROMPT**##

In [ ]:
# @title PROMPT, GUNAKAN BAHASA INGGRIS / USE ENGLISH { display-mode: "form" }
import os
prompt = "1girl" #@param {type:"string"}
seed = 4329492846 # @param {type:"slider", min:0, max:9000000000, step:1}

negative_prompt = "bad-picture-chill-75v, ng_deepnegative_v1_75t, badhandv4, (worst quality:2), (low quality:2), (normal quality:2), (lowres:2), (bad anatomy:2), (bad hands:2), (watermark:2), (mole:1.5), (freckles:1.5)" #@param {type:"string"}

width = 512  #@param {type:"slider", min:8, max:2048, step:8}
height = 512  #@param {type:"slider", min:8, max:2048, step:8}
num_image = 1 #@param {type:"slider", min:1, max:20}
similarity2prompt = 7.5 #@param {type:"slider", min:0.5, max:10, step:0.5}
step = 50 #@param {type:"slider", min:1, max:100}

# Mengambil nilai dari widget param
width = int(width)
height = int(height)
image = int(num_image)
guidance = float(similarity2prompt)
step = int(step)

images = pipe(
    prompt=prompt,
    width=width,
    height=height,
    num_images_per_prompt = image,
    guidance_scale = guidance,
    num_inference_steps = step,
    negative_prompt=negative_prompt,
    output_type="latent" if use_refiner else "pil",
    generator=torch.Generator().manual_seed(seed)
).images


if use_refiner:
  images = refiner(
      prompt = prompt,
      negative_prompt = negative_prompt,
      image = images,
      ).images

print(f"Prompt:\t{prompt}\nSeed:\t{seed}")

# Nama file yang akan digunakan
base_filename = "output.jpg"
new_filename = base_filename

# Cek apakah file dengan nama yang sama sudah ada
if os.path.exists(base_filename):
    # Jika sudah ada, tambahkan angka di belakangnya
    index = 1
    while True:
        new_filename = f"output_{index}.jpg"
        if not os.path.exists(new_filename):
            break
        index += 1

# Simpan gambar dengan nama yang telah ditentukan
images[0].save(new_filename)

# Menampilkan gambar
media.show_images(images)

## **TOOLS**

In [ ]:
#@title FIX FALSE NSFW
pipe.safety_checker = None
pipe.requires_safety_checker = False

#JIKA RUNTIME RESTART, JALANKAN KODE INI LAGI
#IF THE RUNTIME RESTARTS, RUN THIS CODE AGAIN